In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from load_dataset import load_data
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

## Instancier le jeu de données

In [16]:
X, y, cat_features = load_data().get_data_X_y(data='simplify')

In [17]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## GaussianNB avec Optimisation

In [18]:
# Définir les hyperparamètres à tester
parameters = {'var_smoothing': [1e-1, 0.5e-1, 1e-3, 0.5e-3, 1e-5, 0.5e-5, 1e-7, 0.5e-7, 1e-9]}

# Initialiser le modèle Naïf Bayes Gaussien
naive_bayes = GaussianNB()

# Utiliser GridSearchCV pour tester différents hyperparamètres
grid_search = GridSearchCV(naive_bayes, parameters, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)

# Obtenir les résultats de la validation croisée
results = pd.DataFrame(grid_search.cv_results_)

# Afficher un tableau des résultats pour chaque combinaison d'hyperparamètres testée
print("Tableau des performances pour différentes combinaisons d'hyperparamètres :")
print(results[['param_var_smoothing', 'mean_test_score', 'std_test_score']])

Tableau des performances pour différentes combinaisons d'hyperparamètres :
  param_var_smoothing  mean_test_score  std_test_score
0                 0.1         0.852577        0.008585
1                0.05         0.857207        0.009911
2               0.001         0.871392        0.011246
3              0.0005         0.870368        0.012424
4             0.00001         0.867265        0.010439
5            0.000005         0.866907        0.010408
6                 0.0         0.866907        0.010408
7                 0.0         0.866907        0.010408
8                 0.0         0.866907        0.010408


## GaussianNB avec les meilleurs hyperparamètres 

In [19]:
naive_bayes_opt = GaussianNB(var_smoothing=0.0005)
naive_bayes_opt.fit(X_train, y_train)

print(f"Train Accuracy : {naive_bayes_opt.score(X_train, y_train) * 100:.3f} %")

Train Accuracy : 84.337 %


In [20]:
print(f"Test Accuracy : {naive_bayes_opt.score(X_test, y_test) * 100:.3f} %")

Test Accuracy : 84.298 %


In [21]:
preds = naive_bayes_opt.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.83      0.73      0.78       414
           1       0.85      0.91      0.88       675

    accuracy                           0.84      1089
   macro avg       0.84      0.82      0.83      1089
weighted avg       0.84      0.84      0.84      1089



## Feature Importance

Le modèle Naive Bayes n'est pas très informatif sur les features et leur importances.

GaussianNB n'offre pas de méthode intrinsèque pour évaluer l'importance des caractéristiques. Les méthodes de Naïve Bayes fonctionnent en déterminant les probabilités conditionnelles et inconditionnelles associées aux caractéristiques et en prédisant la classe avec la probabilité la plus élevée. Ainsi, aucun coefficient n'est calculé ou associé aux caractéristiques qu'on utilise pour entraîner le modèle.

## Conclusion

Le modèle donne quand même un bon score, mais il est peu informatif et dûr à expliquer à partir des features.